### **Practise I**

**Sensor Status Report**

We will create an application and a `/sensor_status` route in it. This route will pass a list of several sensors to the `sensors.html` HTML template. Each sensor will be a dictionary with `name` and `status` keys.

- In the `sensors.html` template, we will then use JINJA to loop through all the sensors and print their names.
- If a sensor has a `status` set to `"offline"`, we will display a **"WARNING: OFFLINE"** warning next to its name.

**Data:**
```python
sensor_data = [
        {'name': 'Core temperature', 'status': 'online'},
        {'name': 'Cabin pressure', 'status': 'online'},
        {'name': 'Gyroscope', 'status': 'offline'},
        {'name': 'Solar panels', 'status': 'online'}
    ]
```

In [ ]:
from flask import Flask, render_template

sensor_data = [
        {'name': 'Core temperature', 'status': 'online'},
        {'name': 'Cabin pressure', 'status': 'online'},
        {'name': 'Gyroscope', 'status': 'offline'},
        {'name': 'Solar panels', 'status': 'online'}
    ]

app = Flask(__name__, template_folder="templates", static_folder="static")

@app.route("/sensor_status")
def sensor_status():
    data = sensor_data
    return render_template("sensors.html", data=data)
# file sensors.html must be in folder "templates"

if __name__ == "__main__":
    app.run(debug=True)

In [ ]:
<!-- sensors.html -->

<h2>Sensors information:</h2>
    {% for sensor in data %}
        <h3>{{ sensor["name"] }}</h3> 
        {% if sensor["status"] == "offline" %} 
            <h4>WARNING: OFFLINE</h4> 
        {% else %}
            <p>Sensor OK</p>
        {% endif %}

    {% endfor %}

### **Practise II**
**Standardized Report Format**

We will create a standardized format for mission reports using template inheritance.

1.  **Create `base_report.html`:** This will be the base (parent) template. It must contain:
    * A heading `<h1>Mission Report</h1>`
    * A block for content
    * A footer `<p>-- End of Transmission --</p>`

2.  **Create `daily_log.html`:** This will be the inheriting (child) template.
    * At the beginning, it must "inherit" everything from `base_report.html`.
    * Then, it must overwrite the content of the block with its own text, for example: `<h2>Daily Log: Day 5</h2><p>Solar wind levels are stable.</p>`.

3.  **In `my_app.py`**, create a simple route `/daily_log` that only displays the `daily_log.html` template.

In [ ]:
from flask import Flask, render_template

app = Flask(__name__, template_folder="templates")

@app.route("/daily_log")
def daily_log():
    return render_template("daily_log.html")

if __name__ == "__main__":
    app.run(debug=True)

In [ ]:
<!-- base_report.html -->

<h1>Mission Report</h1>

{% block content %}
    <p>blok pro přepsání</p>
{% endblock content %}

<p>-- End of Transmission --</p>

In [ ]:
<!-- daily_log.html -->
 
{% extends "base_report.html" %}

{% block content %} 
    <h2>Daily Log: Day 5</h2>
    <p>Solar wind levels are stable.</p> 
{% endblock content %}

### **Project (Homework): Central Data Hub Implementation**

**Mission:** Enhance the information panel from the previous mission with a unified structure and dynamic display of crew data. The goal is to apply knowledge of the Jinja2 templating system and increase the application's modularity.

`Basic application structure`:
- my_project/
  - my_app.py
  - **routes.py**
  - templates/
  - static/
  - **config/**

1.  **Creating a Standardized Interface (Master Template):**
- Create a base template `base.html` that will define a unified look for the entire information panel.
- Create partial templates `header.html` (for the navigation bar with links) and `footer.html` (for your corporation's copyright footer).
- Using `{% include ... %}`, insert `header.html` and `footer.html` into the base template `base.html`.
- In `base.html`, define at least one block for the main page content, e.g., `{% block content %}`.

2.  **Upgrading Existing Panels:**
- Modify all your templates for the routes (`/`, `/mission_briefing`, `/target_asteroid`) so that they inherit from `base.html`.
- Insert the unique content of the templates into the `content` block.

3.  **Upgrading Crew Info:**
- From the existing `/crew_members/<int:id>` route, extract the list of dictionaries with crew data and make it a global variable.
- Further modify the `/crew_members/<int:id>` route to generate an HTML page using `render_template()`.
- Then, create a new route and template for `/crew_members`.
- In the template for the `/crew_members` route, use a loop to display the names of all crew members - each name will also be a **link** - use `url_for()`.
- This link (for each crew member) will lead to their detailed profile at `/crew_members/<int:id>`.

4.  **Upgrading Asteroid Information:**
- Remove the route and HTML file for `/target_asteroid/<string:asteroid_name>`.
- For the `/target_asteroid/` route, move the data from the HTML file to a global variable in our program and pass it to the HTML.

5.  **Optimizing Communication Protocols (Bonus I):**
- Move all your route definitions to a separate file (e.g., `routes.py`) and import them into your main program.

6.  **Unified Configuration (Bonus II):**
- Move the configuration from the application to a file in the `config/` folder.

In [ ]:
# configuration.py

class Config:
    """
    Set Flask config variables
    """
    # General Config
    DEBUG = True # setting for debugging

In [ ]:
from flask import Flask
from config.configuration import Config # import "Config" class from "configuration" file in "config" folder
from routes import register_routes

my_crew = [
        {
            "id": 1,
            "name": "Dave Fisher",
            "specialization": "AI Engineer",
            "status": "Active"
        },
        {
            "id": 2,
            "name": "Izael Alexander",
            "specialization": "Rocket Scientist",
            "status": "Active"
        },
        {
            "id": 3,
            "name": "Tethra Dyagran",
            "specialization": "Roboticist",
            "status": "Active"
        },
        {
            "id": 4,
            "name": "Mura Lan",
            "specialization": "Astronavigator",
            "status": "Active"
        }
    ]

my_asteroid = {"name": "230 Athamantis",
               "diameter": "118±2 km",
               "mass": "(2.3±1.1)*10**18 kg",
               "density": "2.7±1.3 g/cm3",
               "speed": "19.3 km/s",
               "img": "static/img/Athamantis.jpeg"
}


app = Flask(__name__)

# -------------- CONFIGURATION --------------
# from a file using a "class"
app.config.from_object(Config)

# -------------- INITIALIZATION --------------
register_routes(app, my_crew, my_asteroid)


# -------------- START appky --------------
if __name__ == "__main__":
    app.run()

In [ ]:
# routes.py
from flask import render_template

def register_routes(app, my_crew, my_asteroid):
    @app.route("/")
    def homepage():
        return render_template("homepage.html")

    @app.route("/mission_briefing")
    def mission_briefing():
        return render_template("mission_briefing.html")

    @app.route("/target_asteroid")
    def target_asteroid():
        return render_template("target_asteroid.html", my_asteroid=my_asteroid)

    @app.route("/crew_members")
    def crew_members():
        return render_template("crew_members.html", my_crew=my_crew)

    @app.route("/crew_members/<int:id>")
    def crew_members_details(id):
        for crew_member in my_crew:
            if crew_member["id"] == id:
                return render_template("crew_members_details.html", crew_member=crew_member)
        return "Crew member not found"

## /templates

In [ ]:
<!-- header.html -->
 
<nav>
    <ul> <!-- all routes (and functions) have to exist for every page we want to create a link for -->
        <li><a href="{{ url_for('homepage') }}">Home</a></li> 
        <li><a href="{{ url_for('mission_briefing') }}">Our Mission</a></li>
        <li><a href="{{ url_for('target_asteroid') }}">The Asteroid</a></li>
        <li><a href="{{ url_for('crew_members') }}">Our Crew</a></li>
    </ul>
</nav>

In [ ]:
<!-- footer.html -->
 
<footer>
    <br>
    <p>Deep Space Exploration Corp.</p>
    <p>All rights reserved</p>
</footer>

In [ ]:
<!-- base.html -->
 
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>{% block page_title %} Title {% endblock page_title %}</title> <!-- dynamic content -->
    
</head>

<body>
    {% include 'header.html' %} <!-- we include the content of "header.html"  -->
    {% block page_content %} Content {% endblock page_content %} <!-- dynamic content -->
    {% include 'footer.html' %} <!-- we include the content of "footer.html" -->
</body>

</html>

In [ ]:
<!-- homepage.html -->

{% extends "base.html" %} <!-- we inherit content of "base.html" -->

{% block page_title %} HomePage {% endblock page_title %}

{% block page_content %}
    <h1>Deep Space Exploration Corp.</h1>
    <p>Welcome to our homepage !</p>
    <p>Our goal is to mine asteroids from the solar system.</p>
{% endblock page_content %}

In [ ]:
<!-- mission_briefing.html -->

{% extends "base.html" %} <!-- we inherit content of "base.html" -->

{% block page_title %} Mission Debriefing {% endblock page_title %}

{% block page_content %}
    <h1>Mission</h1>
    <p>This is our mission:</p>
    <p>get resources from chosen asteroids</p>
    <p>make it profitable</p>
    <p>expand world's economy</p>
{% endblock page_content %}

In [ ]:
<!-- target_asteroid.html -->

{% extends "base.html" %} <!-- we inherit content of "base.html" -->

{% block page_title %} Target Asteroid {% endblock page_title %}

{% block page_content %}
    <h1>Target asteroid: {{ my_asteroid["name"] }}</h1>
    <p>Mean diameter: {{ my_asteroid["diameter"] }}</p>
    <p>Mass: {{ my_asteroid["mass"] }}</p>
    <p>Mean density: {{ my_asteroid["density"] }}</p>
    <p>Average orbital speed: {{ my_asteroid["speed"] }}</p>
    <img src="{{ my_asteroid['img'] }}" width="800px"/>  
{% endblock page_content %}

In [ ]:
<!-- crew_members_details.html -->

{% extends "base.html" %} <!-- we inherit content of "base.html" -->

{% block page_title %} {{ crew_member["name"] }} {% endblock page_title %}

{% block page_content %}
    <h1>Crew member {{ crew_member["id"] }}</h1>
    <p>Name: {{ crew_member["name"] }}</p>
    <p>Specialization: {{ crew_member["specialization"] }}</p>
    <p>Status: {{ crew_member["status"] }}</p>
{% endblock page_content %}

In [ ]:
<!-- crew_members.html -->

{% extends "base.html" %} <!-- we inherit content of "base.html" -->

{% block page_title %} Our Crew {% endblock page_title %}

{% block page_content %}
    {% for member in my_crew %}
        <p><a href="{{ url_for('crew_members_details', id=member['id']) }}">{{ member["name"] }}</a></p>
    {% endfor %}
{% endblock page_content %}

---
#### © Jiří Svoboda (George Freedom)
- Web: https://GeorgeFreedom.com
- LinkedIn: https://www.linkedin.com/in/georgefreedom/
- Book me: https://cal.com/georgefreedom